**Cartographie Haute Résolution du Carbone Organique du Sol (Algérie)**

Instructeur : Dr. Ghiles Kaci

Durée : 2 Heures

Zone Géographique : Atlas Tellien, Algérie

**1. Introduction et Objectifs d'Apprentissage**

**Le Défi :**
Les cartes mondiales des sols (comme SoilGrids) sont excellentes, mais elles sont souvent trop grossières (résolution de 250m) pour la prise de décision agricole locale. Les agriculteurs gèrent leurs terres à l'échelle de mètres, pas de centaines de mètres.

**La Solution :**
Nous pouvons effectuer un « downscaling » (réduction d'échelle) de ces cartes grossières en les fusionnant avec des données satellitaires à haute résolution. Nous utilisons la carte grossière comme « enseignant » (données d'entraînement) et les satellites à haute résolution (Landsat, SRTM) comme « prédicteurs » pour générer une nouvelle carte détaillée à 30m.

**Objectifs d'Apprentissage :**



*   **Accéder** aux données géospatiales cloud-natives (Landsat, SRTM) et aux données locales des sols.
*   **Calculer** des indices environnementaux (MSAVI, Indice de Sol Nu) pour représenter la végétation et la santé des sols.
*   **Construire** un jeu de données d'entraînement par rééchantillonnage et alignement de rasters disparates (Downscaling).
*   **Entraîner** un régresseur Random Forest pour prédire le Carbone Organique du Sol (COS).
*   **Quantifier** l'incertitude du modèle en utilisant une approche d'ensemble.

**2. Configuration et Bibliothèques**

Nous nous appuyons sur la pile Pangeo — un écosystème moderne d'outils open source pour les grandes données géospatiales.

1. xarray / rioxarray : Pour manipuler les données raster multidimensionnelles (tableaux étiquetés).

2. pystac_client : Pour rechercher dans les catalogues satellites comme Google.

3. scikit-learn : Pour le modèle d'apprentissage automatique Random Forest.

In [55]:
# Installer les packages nécessaires (décommenter si exécution dans Colab/Binder)
!pip install pystac-client planetary-computer rioxarray xarray geopandas rasterio scikit-learn matplotlib

import pystac_client
import planetary_computer
import rioxarray
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Configurer matplotlib pour de jolis graphiques
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')


**3. Acquisition des Données**

**3.1 Définir la Zone d'Intérêt (ZI)**

Pour assurer une exécution fluide sur vos ordinateurs portables, nous allons nous concentrer sur une fenêtre agricole spécifique dans la région de l'Atlas Tellien du nord de l'Algérie. Nous définissons cela en utilisant une Boîte Englobante (Bounding Box).

In [56]:
# Définir la Zone d'Intérêt (Atlas Tellien, Algérie)
# Format : [lon_min, lat_min, lon_max, lat_max]
# Ceci couvre une zone d'environ 10km x 10km pour assurer un traitement rapide
aoi_bbox = [3.0, 36.4, 3.15, 36.5]

print(f"Zone d'étude définie : {aoi_bbox}")


**3.2 Accéder à la Variable Cible (SoilGrids)**

Nous utilisons **SoilGrids (ISRIC)** comme notre « Pseudo-Vérité Terrain ». Nous nous intéressons au stock de COS (0-5cm).

**Gestion de l'Accès aux Données :**
Nous allons charger le fichier local out.tif qui contient les données SoilGrids. Le code ci-dessous est robuste : il vérifie à la fois votre dossier actuel (si vous l'avez téléchargé) et votre chemin Windows local.

In [57]:
# Définir le nom du fichier
target_file = "out.tif"

# Liste des chemins à vérifier
# 1. Vérifier le répertoire actuel (par défaut pour les téléchargements Colab/Cloud)
# 2. Vérifier votre chemin Windows local spécifique (si exécution locale)
possible_paths = [
    target_file,
    r"C:\Users\ghile\Downloads\out.tif"
]

local_filepath = None
for path in possible_paths:
    if os.path.exists(path):
        local_filepath = path
        print(f"Succès ! Fichier trouvé à : {local_filepath}")
        break

if local_filepath is None:
    # Informations de débogage si le fichier est manquant
    print(f"\nERREUR : Impossible de trouver '{target_file}' dans ces emplacements :")
    for p in possible_paths:
        print(f" - {p}")
    print(f"\nRépertoire de travail actuel : {os.getcwd()}")
    try:
        print(f"Fichiers disponibles ici : {os.listdir()}")
    except:
        pass

    raise FileNotFoundError("Veuillez télécharger 'out.tif' dans l'environnement de ce notebook (onglet Fichiers à gauche).")

# Ouvrir le fichier trouvé
soc_coarse = rioxarray.open_rasterio(local_filepath)

# --- POST-TRAITEMENT ---
# Découper selon la ZI (Vérification de sécurité)
soc_coarse = soc_coarse.rio.clip_box(*aoi_bbox)

# Renommer en 'soc' et sauvegarder en mémoire
soc_coarse.name = "soc"

# Visualiser la « Vérité »
soc_coarse.plot(cmap="magma")
plt.title("Cible : Stock COS Grossier (0-5cm) [SoilGrids 250m]")
plt.show()

print("Résolution COS originale :", soc_coarse.rio.resolution())


**3.3 Accéder aux Prédicteurs : Relief (Topographie)**

La topographie contrôle l'écoulement de l'eau et l'accumulation des sols. Nous utilisons le **NASADEM (SRTM)** à résolution 30m.

In [58]:
# Se connecter au Catalogue STAC de Microsoft Planetary Computer
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Rechercher les Données d'Élévation
search = catalog.search(
    collections=["nasadem"],
    bbox=aoi_bbox
)
item = next(search.items()) # Obtenir la première tuile disponible

# Charger l'Élévation
elevation = rioxarray.open_rasterio(item.assets["elevation"].href)
elevation = elevation.rio.clip_box(*aoi_bbox)
# Supprimer la dimension 'band', car nous travaillons généralement avec une seule bande pour l'élévation
elevation = elevation.squeeze('band', drop=True)

# Calculer la Pente (Gradient numérique simplifié)
# Dans un flux de production complet, nous utiliserions xarray-spatial pour cela
# Maintenant elevation.values est 2D, et elevation.coords/dims sont 2D
dx, dy = np.gradient(elevation.values)
slope = np.sqrt(dx**2 + dy**2)
slope_da = xr.DataArray(slope, coords=elevation.coords, dims=elevation.dims)
slope_da.name = "slope"

# Afficher
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
elevation.plot(ax=ax1, cmap="terrain")
ax1.set_title("Élévation (m)")
slope_da.plot(ax=ax2, cmap="Greys")
ax2.set_title("Gradient de Pente")
plt.show()


**3.4 Accéder aux Prédicteurs : Imagerie Satellitaire (Landsat)**

Nous utilisons les bandes Landsat pour dériver des indices de végétation et de sol.

In [59]:
# Rechercher Landsat Collection 2 Niveau-2 (Réflectance de Surface)
search_landsat = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=aoi_bbox,
    # Plage de dates large (Printemps/Été 2023-2024) pour trouver de bonnes données
    datetime="2023-01-01/2024-12-31",
    query={"eo:cloud_cover": {"lt": 20}} # Assoupli à 20% pour s'assurer de trouver une scène
)

# Obtenir tous les éléments trouvés et trier par couverture nuageuse (moins nuageux en premier)
landsat_items = sorted(list(search_landsat.items()), key=lambda item: item.properties["eo:cloud_cover"])

print(f"{len(landsat_items)} scènes Landsat potentielles trouvées.")

if not landsat_items:
    raise ValueError("Aucune scène Landsat trouvée. Essayez d'augmenter le seuil de couverture nuageuse ou d'élargir la plage de dates.")

# Essayer de trouver une scène qui couvre complètement notre ZI
selected_item = None
bands = {}
required_bands = ["red", "blue", "nir08", "swir16"]

print("Vérification des scènes pour une couverture complète...")
for item in landsat_items:
    try:
        current_bands = {}
        # Essayer de charger toutes les bandes requises
        for band in required_bands:
            da = rioxarray.open_rasterio(item.assets[band].href)

            # --- CORRECTION CRITIQUE ---
            # Les données Landsat sont en UTM (mètres), mais notre bbox est en Lat/Lon (degrés).
            # Nous DEVONS indiquer à clip_box que nos coordonnées sont EPSG:4326 pour qu'il puisse les transformer.
            clipped_da = da.rio.clip_box(*aoi_bbox, crs="EPSG:4326")

            # Vérifier si des données valides existent (pas seulement des pixels nodata vides)
            if clipped_da.count() == 0:
                # Cela peut arriver si la boîte reprojetée tombe en dehors des limites de l'image
                raise ValueError("Données vides après découpage")

            current_bands[band] = clipped_da

        # Si nous arrivons ici, toutes les bandes ont été chargées et découpées avec succès
        selected_item = item
        bands = current_bands
        print(f"Scène sélectionnée : {selected_item.id} | Couverture nuageuse : {selected_item.properties['eo:cloud_cover']}%")
        break

    except Exception as e:
        # Si une bande échoue (ex. ZI au bord de la scène), passer à l'élément suivant
        continue

if selected_item is None:
    raise ValueError("Impossible de trouver une scène unique couvrant entièrement la ZI. Veuillez ajuster les coordonnées de la ZI.")

# --- CALCULER LES INDICES ---

# 1. Formule MSAVI : (2 * NIR + 1 - sqrt((2 * NIR + 1)^2 - 8 * (NIR - Red))) / 2
# Note : Utiliser les valeurs DN brutes pour les indices basés sur des ratios est une simplification courante en enseignement.
# Idéalement, appliquer les facteurs d'échelle (0.0000275 + -0.2) pour les valeurs physiques.
nir = bands["nir08"].astype(float)
red = bands["red"].astype(float)
blue = bands["blue"].astype(float)
swir = bands["swir16"].astype(float)

msavi = (2 * nir + 1 - np.sqrt((2 * nir + 1)**2 - 8 * (nir - red))) / 2
msavi.name = "msavi"

# 2. Indice de Sol Nu (BSI)
# Formule : ((Rouge + SWIR) - (NIR + Bleu)) / ((Rouge + SWIR) + (NIR + Bleu))
bsi = ((red + swir) - (nir + blue)) / ((red + swir) + (nir + blue))
bsi.name = "bsi"

# Visualiser
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
msavi.plot(ax=ax1, cmap="YlGn", vmin=-1, vmax=1)
ax1.set_title("MSAVI (Végétation)")
bsi.plot(ax=ax2, cmap="copper_r", vmin=-1, vmax=1)
ax2.set_title("BSI (Sol Nu)")
plt.show()

**4. Ingénierie des Données (La « Magie » du Downscaling)**

C'est l'étape technique la plus critique. Nous avons :

1. **Cible :** COS à 250m.

2. **Prédicteurs :** Élévation/Landsat à 30m.

Nous devons aligner tout sur la grille 30m. Cela « étale » effectivement les données de sol grossières sur la grille fine, permettant au modèle d'apprentissage automatique d'apprendre la relation entre le motif de sol grossier et les détails environnementaux fins.

In [60]:
# 1. Définir la Grille Maître (La grille Landsat 30m)
master_grid = msavi

# 2. Reprojeter et Rééchantillonner Tout pour Correspondre à la Grille Maître
# Nous utilisons 'nearest' pour la cible pour préserver les valeurs originales sans lissage
soc_aligned = soc_coarse.rio.reproject_match(master_grid, resampling=1) # 1=Plus proche voisin
elevation_aligned = elevation.rio.reproject_match(master_grid)
slope_aligned = slope_da.rio.reproject_match(master_grid)

print(f"Forme et NaN pour les données alignées avant empilement :")
print(f"  soc_aligned : forme={soc_aligned.shape}, NaN={soc_aligned.isnull().sum().item()}")
print(f"  elevation_aligned : forme={elevation_aligned.shape}, NaN={elevation_aligned.isnull().sum().item()}")
print(f"  slope_aligned : forme={slope_aligned.shape}, NaN={slope_aligned.isnull().sum().item()}")
print(f"  msavi (maître) : forme={msavi.shape}, NaN={msavi.isnull().sum().item()}")
print(f"  bsi : forme={bsi.shape}, NaN={bsi.isnull().sum().item()}")


# 3. Empiler tous les Prédicteurs dans un « Cube »
# Note : Nous utilisons squeeze() pour supprimer la dimension 'band' qui a généralement une taille de 1
stack = xr.merge([
    msavi.rename("msavi").squeeze(),
    bsi.rename("bsi").squeeze(),
    elevation_aligned.rename("elevation").squeeze(),
    slope_aligned.rename("slope").squeeze()
])

# Ajouter la Cible (COS) à la pile
stack["soc"] = soc_aligned.squeeze()

print(f"\nForme de la pile après fusion (avant filtre COS) : {stack.dims}")
print("NaN par variable dans la pile (avant filtre COS) :")
for var_name in stack.data_vars:
    print(f"  {var_name} : {stack[var_name].isnull().sum().item()}")

# 4. Masquer les NaN (pixels avec données manquantes) plus soigneusement
original_total_pixels = stack.soc.size
# Filtrer les pixels où COS est <= 0 (valeurs invalides)
stack = stack.where(stack["soc"] > 0)
pixels_after_soc_filter = stack.soc.notnull().sum().item()
print(f"\nPixels avec COS > 0 après filtre : {pixels_after_soc_filter} sur {original_total_pixels}")

# Convertir en DataFrame, puis dropna uniquement pour la colonne 'soc' pour les données d'entraînement
# Cela garantit que nous conservons les pixels pour lesquels nous avons une valeur cible valide,
# même si certaines variables prédictives peuvent avoir des NaN (que Random Forest peut gérer)
# Nous utiliserons ce df directement dans la cellule suivante.


**5. Créer les Données d'Entraînement (Pseudo-Vérité Terrain)**

Nous ne pouvons pas alimenter directement une image dans un Random Forest standard. Nous avons besoin d'une **Table** (DataFrame).
Nous allons échantillonner aléatoirement 10 000 points de notre pile alignée.

In [61]:
# Convertir le Dataset xarray en DataFrame Pandas
# Cela aplatit la carte en une table de pixels
df = stack.to_dataframe().reset_index()

# Supprimer explicitement les lignes où la variable cible 'soc' est NaN
df = df.dropna(subset=["soc"])

# Supprimer également les pixels où msavi, bsi, elevation ou slope sont NaN, car ce sont des caractéristiques
df = df.dropna(subset=["msavi", "bsi", "elevation", "slope"])

print(f"Total de pixels disponibles après nettoyage : {len(df)}")

# Échantillonner 10 000 pixels aléatoires pour l'entraînement
# Nous utilisons un sous-ensemble pour simuler l'échantillonnage de terrain et accélérer l'entraînement
# S'assurer de ne pas essayer d'échantillonner plus que les pixels disponibles
sample_size = min(10000, len(df))
if sample_size == 0:
    raise ValueError("Aucun pixel valide disponible après nettoyage. Impossible de procéder à l'échantillonnage.")
training_data = df.sample(n=sample_size, random_state=42)

print("En-tête de la Table d'Entraînement :")
print(training_data[["soc", "msavi", "bsi", "slope"]].head())

# Division Caractéristiques/Cible
X = training_data[["msavi", "bsi", "elevation", "slope"]]
y = training_data["soc"]

**6. Modélisation : Régresseur Random Forest**

Nous utilisons l'algorithme Random Forest car il gère bien les relations non linéaires (ex. le COS augmente avec la végétation mais pourrait diminuer si les pentes deviennent trop raides).

In [62]:
# Diviser en ensembles d'Entraînement et de Validation (80% entraînement, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser le Modèle
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Entraîner le Modèle
rf_model.fit(X_train, y_train)

# Valider
predictions = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print(f"Performance du Modèle :")
print(f"RMSE : {rmse:.2f} (Mg C/ha)")
print(f"R-Carré : {r2:.2f}")

# Graphique d'Importance des Caractéristiques
importances = pd.Series(rf_model.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', color='teal')
plt.title("Qu'est-ce qui détermine le COS en Algérie ?")
plt.xlabel("Importance")
plt.show()

**7. Cartographie Opérationnelle et Incertitude**

Nous allons maintenant appliquer notre « cerveau » entraîné (le modèle) à toute la zone d'étude (tous les pixels, pas seulement ceux d'entraînement).
Pour quantifier la confiance, nous allons exécuter un petit **Ensemble** : nous prédisons 10 fois, et vérifions où les prédictions concordent (stable) ou divergent (incertain).

In [63]:
# Préparer les données d'entrée complètes pour la prédiction
# Nous devons supprimer la colonne 'soc' et les colonnes de coordonnées
full_input = df[["msavi", "bsi", "elevation", "slope"]]

# --- PRÉDICTION D'ENSEMBLE ---
n_iterations = 10
all_predictions = []

print("Exécution de la Boucle de Prédiction d'Ensemble...")
for i in range(n_iterations):
    # Entraîner un nouveau modèle avec une graine aléatoire différente
    model = RandomForestRegressor(n_estimators=25, random_state=i, n_jobs=-1)
    model.fit(X_train, y_train)

    # Prédire sur la zone COMPLÈTE
    pred = model.predict(full_input)
    all_predictions.append(pred)

# Convertir la liste de tableaux en tableau numpy
all_predictions = np.array(all_predictions) # Forme : (10, num_pixels)

# Calculer la Moyenne (La Carte Finale) et l'Écart-Type (La Carte d'Incertitude)
mean_pred = np.mean(all_predictions, axis=0)
std_pred = np.std(all_predictions, axis=0)

# Remettre dans le dataframe pour reconstruire la carte
df["soc_predicted_mean"] = mean_pred
df["soc_uncertainty"] = std_pred

# Convertir le DataFrame en Xarray pour l'affichage
final_ds = df.set_index(["y", "x"]).to_xarray()


**8. Visualisation Finale**

Examinons notre résultat.

**Gauche :** Notre nouvelle carte COS haute résolution (30m).

**Droite :** Où le modèle est-il confus ? (Incertitude).

In [64]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

# Afficher la Prédiction Moyenne
final_ds["soc_predicted_mean"].plot(ax=ax1, cmap="viridis")
ax1.set_title("Stock COS Prédit (0-5cm) [Rés. 30m]")
ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude")

# Afficher l'Incertitude (Écart-Type)
final_ds["soc_uncertainty"].plot(ax=ax2, cmap="inferno")
ax2.set_title("Incertitude de Prédiction (É-T)")
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")

plt.tight_layout()
plt.show()


**Question de Discussion :**
Observez la Carte d'Incertitude. Voyez-vous une incertitude plus élevée dans les montagnes (forte pente) ou dans les plaines agricoles plates ? Pourquoi le modèle pourrait-il avoir des difficultés dans ces zones ?